# Lab 10 · Case Management and Chain of Custody

This lab demonstrates how to model a minimal case workflow with synthetic data. We create a case via an SDK-inspired helper, register an evidence note, and review the resulting chain of custody entries as a pandas table.

In [ ]:
DRY_RUN = True
from notebooks._utils.common import *

DRY_RUN.value = True
CLI_OK.value = shell_available('forensic-cli')
LAB_ID = '10_case'
LAB_ROOT = lab_root(LAB_ID)


## SDK-style helper for deterministic case handling

The helper stores structured JSON under `.labs/10_case/` so repeated runs remain deterministic.

In [ ]:
from dataclasses import dataclass, field
import hashlib
from pathlib import Path
import pandas as pd

@dataclass
class MiniCaseSDK:
    base: Path
    investigator: str
    coc: dict[str, list[dict]] = field(default_factory=dict)

    def __post_init__(self) -> None:
        self.case_dir = self.base / 'cases'
        self.case_dir.mkdir(parents=True, exist_ok=True)
        self.artifact_dir = self.base / 'artifacts'
        self.artifact_dir.mkdir(parents=True, exist_ok=True)
        self.coc_dir = self.base / 'coc'
        self.coc_dir.mkdir(parents=True, exist_ok=True)

    def create_case(self, case_id: str, description: str) -> dict:
        created_at = _ts()
        record = {
            'case_id': case_id,
            'created_at': created_at,
            'investigator': self.investigator,
            'description': description,
            'status': 'open',
        }
        json_dump_sorted(record, self.case_dir / f'{case_id}.json')
        entry = {
            'timestamp': created_at,
            'actor': self.investigator,
            'action': 'case_created',
            'artifact': 'n/a',
        }
        self.coc[case_id] = [entry]
        self._flush_coc(case_id)
        return record

    def add_text_artifact(self, case_id: str, artifact_name: str, text: str) -> dict:
        if case_id not in self.coc:
            raise ValueError(f'Case {case_id} missing')
        artifact_path = self.artifact_dir / artifact_name
        artifact_path.write_text(text, encoding='utf-8')
        digest = hashlib.sha256(text.encode('utf-8')).hexdigest()
        entry = {
            'timestamp': _ts(),
            'actor': self.investigator,
            'action': 'artifact_added',
            'artifact': artifact_name,
            'sha256': digest,
        }
        self.coc[case_id].append(entry)
        self._flush_coc(case_id)
        return {
            'path': artifact_path,
            'sha256': digest,
        }

    def get_chain_of_custody(self, case_id: str) -> list[dict]:
        return list(self.coc.get(case_id, []))

    def _flush_coc(self, case_id: str) -> None:
        json_dump_sorted(
            {
                'case_id': case_id,
                'entries': self.coc[case_id],
            },
            self.coc_dir / f'{case_id}.json',
        )


## Create a deterministic case and add an artefact

We capture both the case metadata and the artefact hash so the chain of custody remains reproducible.

In [ ]:
sdk = MiniCaseSDK(base=LAB_ROOT, investigator='analyst@example.com')
case_id = 'case-lab-001'
case_record = sdk.create_case(case_id, description='Synthetic login anomaly review')
artifact_details = sdk.add_text_artifact(
    case_id,
    artifact_name=f'{case_id}_note.txt',
    text='Observed unusual access pattern from 203.0.113.10 at 09:02Z.',
)
case_record, artifact_details


## Review artefact preview and hash

Hashes are essential for maintaining integrity during evidence handling.

In [ ]:
print('Artefact preview:')
print(preview(artifact_details['path']))
print('
SHA256:', artifact_details['sha256'])


## Display chain of custody entries

We render the entries with pandas to obtain a clean tabular view.

In [ ]:
coc_entries = sdk.get_chain_of_custody(case_id)
coc_df = pd.DataFrame(coc_entries)
coc_df


## Optional CLI mirror

If the `forensic-cli` is installed locally, mirror the inspection of the case ledger.

In [ ]:
if CLI_OK.value:
    rc, out, err = run_cli(
        ['forensic-cli', 'case', 'show', '--case-id', case_id],
        tolerate=True,
    )
    print('return code:', rc)
    print(out or err)
else:
    print('CLI mirror skipped; forensic-cli not available in this environment.')


## Checkpoint

Ensure that both the artefact and the chain-of-custody ledger are present. This guard helps when running inside CI.

In [ ]:
assert artifact_details['path'].exists()
assert (LAB_ROOT / 'coc' / f'{case_id}.json').exists()
len(coc_entries)
